<a href="https://colab.research.google.com/github/mohamedfadul/DeepSearch-Finding-Truncated-Differential-Trails-Using-Genetic-Algorithms-and-Deep-Learning/blob/main/ANN_Generalization_model_using_data_up_to_round_8_to_predict_TWINE_round_15_Trail_Validity_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use seaborn for pairplot
!pip install -q seaborn

# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [2]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.12.0


In [4]:
#import tensorflow_docs as tfdocs
#import tensorflow_docs.plots
#import tensorflow_docs.modeling

In [5]:
!wget -O all_dataset_path.csv "https://drive.google.com/uc?export=download&id=12q2SgUC5Q0BEz0qVg5rtlSaivGMsMH74"

--2023-07-17 07:58:35--  https://drive.google.com/uc?export=download&id=12q2SgUC5Q0BEz0qVg5rtlSaivGMsMH74
Resolving drive.google.com (drive.google.com)... 173.194.218.138, 173.194.218.113, 173.194.218.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ftphj2vht4b879sau90j499ovl24hksl/1689580650000/14347683118445347781/*/12q2SgUC5Q0BEz0qVg5rtlSaivGMsMH74?e=download&uuid=c6629a41-e68b-4f52-88f0-500424e7144c [following]
--2023-07-17 07:58:35--  https://doc-0k-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ftphj2vht4b879sau90j499ovl24hksl/1689580650000/14347683118445347781/*/12q2SgUC5Q0BEz0qVg5rtlSaivGMsMH74?e=download&uuid=c6629a41-e68b-4f52-88f0-500424e7144c
Resolving doc-0k-54-docs.googleusercontent.com (doc-0k-54-docs.googleusercontent.com)... 108.177.13.132, 2

In [6]:
test_dataset =pd.read_csv ('all_dataset_path.csv')
test_dataset

,input16,input15,input14,input13,input12,input11,input10,input9,input8,input7,...,perm7,perm8,perm9,perm10,perm11,perm12,perm13,perm14,perm15,perm16
0,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
1,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
2,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
3,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
4,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,1,1,1,1,1,1,1,0,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
534,1,1,1,1,1,1,1,1,0,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
535,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
536,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333


In [7]:
test_labels = test_dataset.pop('validity')

In [8]:
test_dataset['sboxes'] = ((test_dataset['sboxes']/(test_dataset['round']*8)))
test_dataset['sboxes']

0      0.091667
1      0.441667
2      2.566667
3      0.741667
4      1.233333
         ...   
533    0.900000
534    0.191667
535    2.791667
536    1.100000
537    1.616667
Name: sboxes, Length: 538, dtype: float64

In [9]:
test_dataset['round'] = (test_dataset['round'])/100

In [10]:
test_dataset

,input16,input15,input14,input13,input12,input11,input10,input9,input8,input7,...,perm7,perm8,perm9,perm10,perm11,perm12,perm13,perm14,perm15,perm16
0,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
1,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
2,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
3,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
4,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,1,1,1,1,1,1,1,0,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
534,1,1,1,1,1,1,1,1,0,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
535,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333
536,1,1,1,1,1,1,1,1,1,1,...,0.2,0.533333,0.866667,0.4,0.6,0.133333,1,0.666667,0.733333,0.933333


In [11]:
from keras.layers import Dropout
from keras import regularizers
from keras.regularizers import l2, l1
reg_param=0.001

def create_model():
  model = keras.Sequential([
        layers.Dense(512, activation='relu' , input_shape=[50]),
        #layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        #layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        #layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        #layers.Dropout(0.2),
      layers.Dense(1,activation='sigmoid' )
    ])
  optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)
  loss = tf.losses.mape
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.optimizers.Adam(learning_rate=0.001), metrics=(['accuracy'], ['Precision'],['Recall']))
  return model

In [12]:
validity_model = create_model()

In [13]:
validity_file_id = '1e2T7UBYXFRL31NYZCr7oM4TxP7HdCTmW'  # Replace 'your_file_id' with the actual file ID

!wget -O validity_model_path.h5 "https://drive.google.com/uc?export=download&id={validity_file_id}"

--2023-07-17 07:58:36--  https://drive.google.com/uc?export=download&id=1e2T7UBYXFRL31NYZCr7oM4TxP7HdCTmW
Resolving drive.google.com (drive.google.com)... 173.194.218.138, 173.194.218.113, 173.194.218.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3sl6ukg2a8ti3t6v0r06ajd8gaf9iu3c/1689580650000/14347683118445347781/*/1e2T7UBYXFRL31NYZCr7oM4TxP7HdCTmW?e=download&uuid=5118e33a-a537-4c43-93ae-2f9545523f2d [following]
--2023-07-17 07:58:39--  https://doc-04-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3sl6ukg2a8ti3t6v0r06ajd8gaf9iu3c/1689580650000/14347683118445347781/*/1e2T7UBYXFRL31NYZCr7oM4TxP7HdCTmW?e=download&uuid=5118e33a-a537-4c43-93ae-2f9545523f2d
Resolving doc-04-54-docs.googleusercontent.com (doc-04-54-docs.googleusercontent.com)... 108.177.13.132, 2

In [14]:
validity_model.load_weights('validity_model_path.h5')

In [15]:
predictions = validity_model.predict(test_dataset)

17/17 [==============================] - 0s 6ms/step


In [16]:
validity_model.evaluate(test_dataset,test_labels)

17/17 [==============================] - 1s 10ms/step - loss: 0.1706 - accuracy: 0.9368 - precision: 0.9091 - recall: 0.7547


[0.17062081396579742,
 0.9368029832839966,
 0.9090909361839294,
 0.7547169923782349]

In [17]:
from sklearn.metrics import confusion_matrix , classification_report
y_preds = validity_model.predict(test_dataset)
y_preds = np.round(y_preds)

print("Classification Report: \n", classification_report(test_labels, y_preds))

17/17 [==============================] - 0s 8ms/step
Classification Report: 
               precision    recall  f1-score   support

           0       0.94      0.98      0.96       432
           1       0.91      0.75      0.82       106

    accuracy                           0.94       538
   macro avg       0.93      0.87      0.89       538
weighted avg       0.94      0.94      0.93       538



In [18]:
predictions

array([[1.03789956e-04],
       [5.44713857e-03],
       [7.33618710e-18],
       [9.53889489e-01],
       [2.90277554e-03],
       [4.83921341e-14],
       [1.65044679e-03],
       [1.21979013e-19],
       [4.51256141e-18],
       [1.08216154e-05],
       [4.23980087e-01],
       [7.02865727e-05],
       [8.36725491e-13],
       [9.94783461e-01],
       [9.81711686e-01],
       [9.44837183e-03],
       [3.05794507e-01],
       [1.07769640e-02],
       [1.24980755e-12],
       [4.71012725e-04],
       [1.82915993e-08],
       [2.16763343e-11],
       [8.30886330e-19],
       [4.15632695e-10],
       [1.24907995e-17],
       [1.91129573e-07],
       [2.06691074e-15],
       [1.88989416e-01],
       [2.49065863e-07],
       [2.34937308e-12],
       [4.76885961e-11],
       [2.06107877e-13],
       [4.32059727e-16],
       [1.09648088e-03],
       [1.33504131e-04],
       [9.89999592e-01],
       [9.92973268e-01],
       [9.15228605e-01],
       [2.65125278e-02],
       [3.25921575e-14],


In [19]:
import numpy as np
rounded = [int(np.round(x[0])) for x in predictions]
#rounded= np.array(rounded)
print(rounded)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 

In [20]:
from sklearn.metrics import f1_score
f1_score(test_labels, rounded)

0.8247422680412373